# Customer complaints analysis

In [2]:
# Importing python libraries to be used in the analysis process

import pandas as pd
import spacy
import gensim
import pyLDAvis
import numpy
import matplotlib
!python -m spacy download en_core_web_sm -qq
import emoji
import time

from gensim import corpora
from gensim.utils import simple_preprocess

import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import TruncatedSVD
# from sklearn.decomposition import LatentDirichletAllocation

import itertools


[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


### Importing customer complaints csv file

In [3]:
customer_complaints = pd.read_csv("comcast_consumeraffairs_complaints")

FileNotFoundError: [Errno 2] No such file or directory: 'comcast_consumeraffairs_complaints'

In [ ]:
# Displaying the first five rows of the imported csv file

customer_complaints.head()

In [ ]:
# Filtering the complaints data to display only the elements that contain a complaint and removing the ones that are empty or null

customer_complaints = customer_complaints[pd.notnull(customer_complaints['text'])]


# Converting the 'text' column (containing text for customer complaints) to a list 

complaints_text_list = customer_complaints.text.values.tolist()


# Printing individual customer's complaints on a separate line 
iterator = 0
while len(complaints_text_list) > iterator:
    for individual_complaint in complaints_text_list:
    
    # Printing the complaint that previously contained an emoji, which emoji was removed as well as the complaint after the emoji was removed.
    
        if len(emoji.emoji_list(individual_complaint)) > 0:
            print(emoji.emoji_list(individual_complaint))
            print(emoji.replace_emoji(individual_complaint, replace=""))
            complaints_text_list[iterator] =  emoji.replace_emoji(individual_complaint, replace="")
            print(" ")
            print(" ")
            print("The list has been upadated.")
            print(" ")
            print(iterator, ": ", complaints_text_list[iterator])
            print(" ")
     
        iterator += 1
       

### Preprocessing complaints text

In [ ]:
# Loading nlp pipeline
nlp = spacy.load('en_core_web_sm')

 

# Defining a lemmatizer function

def lemmatizer(list):
    '''
    Lemmatize a list of texts
    '''
    lemmatized_list = []

    doc_w_lemma = [x.lemma_ for x in list]

    return doc_w_lemma

# Defining a text_preprocessor function that uses the lemmatizer function for lemmatization

def text_preprocessor(list):
    '''
    Preprocess a list of texts, removing punctuations, stopwords and lemmatizing the texts
    '''
    processed_list_of_complaints = []
    
    for x in list:
        x = x.lower()  # To convert all text to lowercase
        x = x.replace("!", " ")
        x = x.replace(".", " ")
        x = x.replace("?", " ") # To remove punctuation that is connected with words like "to?Do" etc.

        document = nlp(x) # Creating a document by running individual complaints through the nlp pipeline for tokenizing
        doc_no_punct = [token for token in document if token.is_punct == False ] # Creating a refined list of tokens from the 'document' tokens without any punctuation
        doc_no_stopwords = [token for token in doc_no_punct if token.is_stop == False]  # Further refining list of tokens from 'doc_no_punct' tokens without any stopwords
        doc_not_empty = [token for token in doc_no_stopwords if len(token) != 0]
        lemmatized_doc = lemmatizer(doc_not_empty)
        processed_list_of_complaints.append(lemmatized_doc)
  
    
    return processed_list_of_complaints


 

list_tester = ["Terrible service!Will never go for comcast.", "Worth only $10?", "Going to throw out my subscriptions!", "Had the worst experience with comcast.", "Expensive and not good.", "Coverage is very low.", "frequently suffers from network disconnection.", "I have started looking for other service companies.", "Not recommended for anyone looking for a stable connection.", "Bought it last week, Not at all satisfied.", "new to the service and already have complaints!", "Customer service is not at all cooperative.", "Has been weeks since last complaint was made still my complaint has not been addressed.", "Worst service not to be recommended to anyone.", "Money wasted for such an expensive service.", "Insufficient cabling", "Bad customer support", "No one picks up at call centre", "Looking for alternative better services", "Totally unsatisfied", "All claims made at the time of buying service not fulfilled", "Not good", "Will not recommend to anyone planning to work online", "was not that bad in the beginning, but now is the worst!"]

start_time = time.time()

processed_text = text_preprocessor(list_tester)

end_time = time.time()

time_diff = end_time - start_time

print("Time taken :  ", time_diff)
print("Processed text:  Complete!")
print(processed_text)



In [ ]:
## Trying to use pandas dataframe.

#def text_preprocessor_text_in(text):
#    '''
#    Preprocess a list of texts, removing punctuations, stopwords and lemmatizing the texts
#    '''
#    processed_list_of_complaints = []
    
        
#    for x in text:
#        x = x.lower()  # To convert all text to lowercase
#        x = x.replace("!", " ") # To remove punctuation that is connected with words like "to?Do" etc.
#        x = x.replace(".", " ")
#        x = x.replace("?", " ")
#        document = nlp(x) # Creating a document by running individual complaints through the nlp pipeline for tokenizing
#        doc_no_punct = [token for token in document if token.is_punct == False ] # Creating a refined list of tokens from the 'document' tokens without any punctuation
#        doc_no_stopwords = [token for token in doc_no_punct if token.is_stop == False]  # Further refining list of tokens from 'doc_no_punct' tokens without any stopwords
#        doc_not_empty = [token for token in doc_no_stopwords if len(token) != 0]
#        lemmatized_doc = lemmatizer(doc_not_empty)
#        processed_list_of_complaints.append(lemmatized_doc)

    
#    return processed_list_of_complaints

def text_preprocessor_text_in(x):

    '''
    Preprocess a list of texts, removing punctuations, stopwords and lemmatizing the texts
    '''
    processed_list_of_complaints = []
    x = x.lower()  # To convert all text to lowercase
    x = x.replace("!", " ") # To remove punctuation that is connected with words like "to?Do" etc.
    x = x.replace(".", " ")
    x = x.replace("?", " ")

    document = nlp(x) # Creating a document by running individual complaints through the nlp pipeline for tokenizing
    doc_no_punct = [token for token in document if token.is_punct == False ] # Creating a refined list of tokens from the 'document' tokens without any punctuation
    doc_no_stopwords = [token for token in doc_no_punct if token.is_stop == False]  # Further refining list of tokens from 'doc_no_punct' tokens without any stopwords
    doc_not_empty = [token for token in doc_no_stopwords if len(token) != 0]
    lemmatized_doc = lemmatizer(doc_not_empty)
    processed_list_of_complaints.append(lemmatized_doc)

    return processed_list_of_complaints

st_time = time.time()
customer_complaints['cleaned'] = customer_complaints['text'].apply(lambda x: text_preprocessor_text_in(x))

en_time = time.time()
time_taken = en_time - st_time
print("Time taken :  ", time_taken)
print("Processed text:\n")
customer_complaints.head(10)


In [ ]:
# To create a single list from the processed_text list of lists

processed_text_single_list_of_words = list(itertools.chain(*processed_text))
print(processed_text_single_list_of_words)

In [ ]:
# Creating a dictionary to keep track of word occurences
def word_occurence(list):
    counts = dict()
    words = list

    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    return counts

print( word_occurence(processed_text_single_list_of_words))

In [79]:
gensim.utils.simple_preprocess(list_tester, deacc=False, min_len=2, max_len=15)

TypeError: decoding to str: need a bytes-like object, list found